### Zeichnungsgerät Aufgaben mit Lösungen

Wir benutzen wieder den im Notebook [Automaten](Automaten.ipynb) beschriebenen Automat um einen Zeichnungsgerät zu steuern, und schreiben Python-Code, der Zeichungsanweisungen für unser Zeichnungsgerät generiert.


Ein Gitter soll gezeichnet werden, welches durch folgenden Dict beschrieben ist:
```python
grid_config = {
    'position': (20, 20),  # Position der linken oberen Ecke, 
    'width': 210, 
    'height': 150, 
    'ncol': 7,  # Anzahl Spalten
    'nrow': 5,  # Anzahl Reihen
    'linewidth': 5, 
    'linecolor': 'grey',
}
```
**Aufgabe 1**:
Wir möchten eine Funktion, die
eine Anweisung generiert, die das im Dict `grid_config` spezifizierte Gitter zeichnet.
Wir möchten die Funktion wie folgt aufrufen können.
```python
make_place_cmd(**grid_config)
```

**Aufgabe 2**: Wie beim Schachbrett möchten wir eine Funktion, die
eine Anweisung generiert, die einen Spielstein der Farbe `color` auf das Feld in 
Spalte `col` und
Reihe `row` 
setzt. Wir möchten die Funktion wie folgt aufrufen können.
```python
make_place_cmd(col, row, color, **grid_config, radius=0.8)
```

**Aufgabe 3**: Wie beim Schachbrett möchten wir eine Funktion,
die eine Anweisung generiert, die einen Spielstein vom Feld in 
Spalte `col` und
Reihe `row`
entfernt, indem mit `'e<width>,<height>;' ein entsprechendes Rechteck gelöscht wird.

Wir möchten die Funktion wie folgt aufrufen können.
```python
make_clear_cmd(col, row, **grid_config)
```

In [ ]:
grid_config = {
    'position': (20, 20),  # Position der linken oberen Ecke, 
    'width': 210, 
    'height': 150, 
    'ncol': 7, 
    'nrow': 5, 
    'linewidth': 5, 
    'linecolor': 'grey',
}

In [ ]:
import zeichnungsautomat as ZA

WIDTH = 300
HEIGHT = 200


def read(cmd, drawingBoard=None):
    if drawingBoard is None:
        _, drawingBoard = ZA.get_automat_and_canvas(width=WIDTH, height=HEIGHT)
    drawingBoard.automaton.read(cmd)
    return drawingBoard

In [ ]:
def make_grid_cmd(position, width, height, ncol, nrow, linewidth, linecolor):
    cmds = []
    x0, y0 = position
    dx, dy = width/ncol, height/nrow

    cmds.append(f'l{linewidth};')
    cmds.append(f's{linecolor};')

    # zeichne horizontale Gitterlinien
    for i in range(nrow+1):
        cmds.append('u')
        cmds.append(f'g{x0},{y0 + i*dy};')
        cmds.append('d')
        cmds.append(f'G{width},{0};')

    # zeichne vertikale Gitterlinien
    for i in range(ncol+1):
        cmds.append('u')
        cmds.append(f'g{x0 + i*dx},{y0};')
        cmds.append('d')
        cmds.append(f'G{0},{height};')

    return ''.join(cmds)

In [ ]:
make_grid_cmd(**grid_config)

In [ ]:
read(make_grid_cmd(**grid_config))

In [ ]:
def make_place_cmd(col, row, color,
                   position, width, height, ncol, nrow, linewidth, linecolor, radius=1):
    cmds = []
    x0, y0 = position
    dx, dy = width/ncol, height/nrow
    max_radius = min(dx, dy)/2 - 3
    radius = max_radius*radius

    cmds.append(f'uf{color};')
    cmds.append(f'g{x0},{y0};')
    cmds.append(f'G{col*dx},{row*dy};')  # bewege Stift auf linke obere Ecke des Feldes
    cmds.append(f'G{dx/2},{dy/2};')  # bewege Stift zur Feldmitte
    cmds.append(f'C{radius};')

    return ''.join(cmds)

In [ ]:
make_place_cmd(1, 2, 'blue', **grid_config, radius=0.5)

In [ ]:
db = read(make_grid_cmd(**grid_config))
db

In [ ]:
place_stone1 = make_place_cmd(1, 2, 'blue', **grid_config, radius=0.5)
place_stone2 = make_place_cmd(3, 3, 'red', **grid_config)
place_stone3 = make_place_cmd(4, 2, 'yellow', **grid_config)

In [ ]:
db = read(place_stone1, db)
db = read(place_stone2, db)
db = read(place_stone3, db)

In [ ]:
def make_clear_cmd(col, row, position, width, height, ncol, nrow, linewidth, linecolor):
    cmds = []
    x0, y0 = position
    dx, dy = width/ncol, height/nrow

    cmds.append('u')
    cmds.append(f'g{x0},{y0};')
    cmds.append(f'G{col*dx},{row*dy};')
    cmds.append(f'G{linewidth/2},{linewidth/2};')
    cmds.append('e{},{};'.format(dx-linewidth, dy-linewidth))

    return ''.join(cmds)

In [ ]:
make_clear_cmd(2, 3, **grid_config)

In [ ]:
db = read(make_grid_cmd(**grid_config))
db

In [ ]:
place_stone1 = make_place_cmd(1, 2, 'blue', **grid_config, radius=0.5)
place_stone2 = make_place_cmd(3, 3, 'red', **grid_config)
place_stone3 = make_place_cmd(4, 2, 'yellow', **grid_config)

remove_stone1 = make_clear_cmd(1, 2, **grid_config)
remove_stone2 = make_clear_cmd(3, 3, **grid_config)
remove_stone3 = make_clear_cmd(4, 2, **grid_config)

In [ ]:
db = read(place_stone1, db)
db = read(place_stone2, db)
db = read(place_stone3, db)

In [ ]:
db = read(remove_stone1, db)
db = read(remove_stone2, db)
db = read(remove_stone3, db)